In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import locale
import nltk
import re
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from IPython.display import display
nltk.download("stopwords")
nltk.download("punkt")
locale.setlocale( locale.LC_ALL, 'es_ES.UTF-8') 
plt.rcParams["figure.figsize"] = (16,9)

[nltk_data] Downloading package stopwords to /Users/tim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/tim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
objetos = pd.read_csv("./data/TAXI_objetos_perdidos.csv", encoding="iso-8859-1",sep=";")
display(objetos.shape)
objetos.head()

(14966, 12)

,Nº incidencia,Iniciativa,Tipo objeto,Fecha anotación incidencia,Fecha cierre incidencia,Taxista dado de alta en la app,Contestación taxista a la incidencia,Fecha Contestación taxista a la incidencia,Taxista localizado telefónicamente,Nº de intentos de localización,Estado,Resolución
0,1,Viajero app,Cartera / monedero,01/06/2018 13:07,25/04/2019 21:04,N,NaN,NaN,S,1.0,Cerrada,Desfavorable
1,2,Viajero app,Gafas,01/06/2018 13:23,01/06/2018 14:18,S,S,01/06/2018 14:18,NaN,NaN,Cerrada,Favorable
2,3,Viajero portal web,Teléfono móvil,01/06/2018 14:19,15/05/2019 21:24,S,S,15/05/2019 21:24,NaN,NaN,Cerrada,Desfavorable
3,4,Viajero portal web,Otros: especificar,01/06/2018 15:24,05/06/2018 15:25,S,N,NaN,NaN,NaN,Cerrada,Desfavorable
4,5,Viajero app,Cartera / monedero,01/06/2018 15:35,05/06/2018 15:40,S,N,NaN,NaN,NaN,Cerrada,Desfavorable


In [10]:
objetos.groupby("Tipo objeto").count()[["Iniciativa"]].sort_values("Iniciativa", ascending=False).rename({"Iniciativa": "n"}, axis=1)

,n
Tipo objeto,
Teléfono móvil,3751
Otros: especificar,2407
"Maleta, bolso, paquete",2095
Cartera / monedero,1742
Gafas,1487
Llaves,1064
Ropa,945
Ordenador,622
Accesorio teléfono móvil,389


In [11]:
objetos[objetos["Tipo objeto"] == "Otros: especificar"]

,Nº incidencia,Iniciativa,Tipo objeto,Fecha anotación incidencia,Fecha cierre incidencia,Taxista dado de alta en la app,Contestación taxista a la incidencia,Fecha Contestación taxista a la incidencia,Taxista localizado telefónicamente,Nº de intentos de localización,Estado,Resolución
3,4,Viajero portal web,Otros: especificar,01/06/2018 15:24,05/06/2018 15:25,S,N,NaN,NaN,NaN,Cerrada,Desfavorable
7,8,Viajero portal web,Otros: especificar,02/06/2018 08:36,04/06/2018 11:12,N,NaN,NaN,S,2.0,Cerrada,Desfavorable
8,9,Viajero portal web,Otros: especificar,02/06/2018 11:38,04/06/2018 12:06,S,S,04/06/2018 12:06,NaN,NaN,Cerrada,Desfavorable
17,18,Viajero portal web,Otros: especificar,03/06/2018 15:02,05/06/2018 12:31,N,NaN,NaN,S,1.0,Cerrada,Favorable
18,19,Viajero portal web,Otros: especificar,03/06/2018 16:05,05/06/2018 08:48,N,NaN,NaN,S,2.0,Cerrada,Desfavorable
...,...,...,...,...,...,...,...,...,...,...,...,...
14953,16767,Viajero portal web,Otros: especificar,31/03/2022 09:01,01/04/2022 10:00,N,NaN,NaN,S,1.0,Cerrada,Favorable
14954,16768,Viajero app,Otros: especificar,31/03/2022 10:02,04/04/2022 10:05,S,N,NaN,NaN,NaN,Cerrada,Desfavorable
14956,16770,Viajero portal web,Otros: especificar,31/03/2022 10:38,04/04/2022 10:40,S,N,NaN,NaN,NaN,Cerrada,Desfavorable
14957,16771,Viajero app,Otros: especificar,31/03/2022 11:29,04/04/2022 11:30,N,NaN,NaN,N,1.0,Cerrada,Desfavorable
